In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from common import *
import qgrid
import ccxt

In [ ]:
print(ccxt.exchanges) # print a list of all available exchange classes

In [142]:
import ccxt
polo = ccxt.poloniex({
    'apiKey': cfg.POLONIEX_API_KEY,
    'secret': cfg.POLONIEX_API_SECRET_KEY,
})
gdax = ccxt.gdax({
    'apiKey': cfg.GDAX_API_KEY,
    'secret': cfg.GDAX_API_SECRET_KEY,
    'passphrase': cfg.GDAX_PASSPHRASE
})
binance = ccxt.binance()

exchange = polo
markets = exchange.load_markets()

print(exchange.id, markets)
print(exchange.fetch_order_book(exchange.symbols[0]))
print(exchange.fetch_ticker('ETH/BTC')) # coin/market - base/quote
print(exchange.fetch_trades('LTC/BTC'))

poloniex {'BCN/BTC': {'maker': 0.0015, 'taker': 0.0025, 'precision': {'amount': 8, 'price': 8}, 'limits': {'amount': {'min': 1e-08, 'max': 1000000000}, 'price': {'min': 1e-08, 'max': 1000000000}, 'cost': {'min': 0.0, 'max': 1000000000}}, 'id': 'BTC_BCN', 'symbol': 'BCN/BTC', 'base': 'BCN', 'quote': 'BTC', 'active': True, 'lot': 1e-08, 'info': {'id': 7, 'last': '0.00000043', 'lowestAsk': '0.00000043', 'highestBid': '0.00000042', 'percentChange': '-0.02272727', 'baseVolume': '288.94297694', 'quoteVolume': '643086734.01657045', 'isFrozen': '0', 'high24hr': '0.00000048', 'low24hr': '0.00000041'}}, 'BELA/BTC': {'maker': 0.0015, 'taker': 0.0025, 'precision': {'amount': 8, 'price': 8}, 'limits': {'amount': {'min': 1e-08, 'max': 1000000000}, 'price': {'min': 1e-08, 'max': 1000000000}, 'cost': {'min': 0.0, 'max': 1000000000}}, 'id': 'BTC_BELA', 'symbol': 'BELA/BTC', 'base': 'BELA', 'quote': 'BTC', 'active': True, 'lot': 1e-08, 'info': {'id': 8, 'last': '0.00002350', 'lowestAsk': '0.00002351', '

{'symbol': 'ETH/BTC', 'timestamp': 1514928138678, 'datetime': '2018-01-02T21:22:19.678Z', 'high': 0.06552603, 'low': 0.05325005, 'bid': 0.05785183, 'ask': 0.05785184, 'vwap': None, 'open': None, 'close': None, 'first': None, 'last': 0.05785183, 'change': 0.02840439, 'percentage': None, 'average': None, 'baseVolume': 230241.39245039, 'quoteVolume': 13988.0745037, 'info': {'id': 148, 'last': '0.05785183', 'lowestAsk': '0.05785184', 'highestBid': '0.05785183', 'percentChange': '0.02840439', 'baseVolume': '13988.07450370', 'quoteVolume': '230241.39245039', 'isFrozen': '0', 'high24hr': '0.06552603', 'low24hr': '0.05325005'}}
[{'info': {'globalTradeID': 313279365, 'tradeID': 13931031, 'date': '2018-01-02 21:22:11', 'type': 'sell', 'rate': '0.01683650', 'amount': '0.61000000', 'total': '0.01027026'}, 'timestamp': 1514928131000, 'datetime': '2018-01-02T21:22:11.000Z', 'symbol': 'LTC/BTC', 'id': '13931031', 'order': None, 'type': 'limit', 'side': 'sell', 'price': 0.0168365, 'amount': 0.61, 'cos

In [ ]:
# Available methods
print (dir (ccxt.poloniex ()))

In [ ]:
# Asyncronous
# import asyncio
# import ccxt.async as ccxt

async def print_poloniex_ethbtc_ticker():
    print(await exchange.fetch_ticker('ETH/BTC'))

asyncio.get_event_loop().run_until_complete(print_poloniex_ethbtc_ticker())

In [143]:
# Prices
def get_price_data_fpath(coin, market, exchange_id, period):
    fname = '{:s}_{:s}_{:s}_{:s}.csv'.format(
        exchange_id, coin, market, str(period))
    return os.path.join(cfg.DATA_DIR, fname)

def get_symbol(coin, market):
    return coin+'/'+market

def fetch_chart_data(exchange, coin, market, period, start, end):
    assert period in exchange.timeframes
    data = exchange.fetch_ohlcv(get_symbol(coin, market), period)#, start, end)
    df = pd.DataFrame(data, columns = c.OHLCV_COLUMNS)
    df['time'] = [datetime.datetime.fromtimestamp(t//1000) for t in df['time']]
    df.set_index('time', inplace=True)
    return df    

def fetch_and_save_chart_data(exchange, coin, market, period, start, end):
    print("Downloading:", get_symbol(coin, market))
    df = fetch_chart_data(exchange, coin, market, period, start, end)
    fpath = get_price_data_fpath(coin, market, exchange.id, period)
    df.to_csv(fpath, index=True)
    print("Downloaded rows:", len(df))
    return df

def load_chart_data(exchange_id, coin, market, period):
    fpath = get_price_data_fpath(coin, market, exchange_id, period)
    df = pd.read_csv(fpath, parse_dates=True, index_col='time')
    return df

def download_chart_data(exchange, coins, market, period, start, end):
    for coin in coins:
        _ = fetch_and_save_chart_data(exchange, coin, market, period, start, end)

def load_currency_group_prices(exchange_id, coins, market, period, start, end):
    df = pd.DataFrame()
    for coin in coins:
        symbol = get_symbol(coin, market)
        data = load_chart_data(exchange_id, coin, market, period)
        data = data[(data.index >= start) & (data.index <= end)]
        df[symbol] = data['close']
    df.dropna(inplace=True)
    return df

In [ ]:
# Get historical data
coins = [c.BTC, c.LTC, c.ETH, c.XRP, c.XMR, c.DASH]
market = c.USDT
coin = c.LTC
symbol = get_symbol(coin, market)
period = '30m' #1800
start = datetime.datetime(year=2015, month=9, day=1)
end = datetime.datetime(year=2017, month=12, day=1)

In [ ]:
#df = fetch_chart_data(exchange, coin, market, period, start, end)
#df = fetch_and_save_chart_data(exchange, coin, market, period, start, end)
df = load_chart_data(exchange.id, coin, market, period)
df.head()

In [ ]:
download_chart_data(exchange, coins, market, period, start, end)

In [ ]:
df = load_currency_group_prices(exchange.id, coins, market, period, start, end)
df.head()

In [ ]:
# Trading
print(exchange.fetch_balance())

# sell one ฿ for market price and receive $ right now
print(exchange.id, exchange.create_market_sell_order('BTC/USD', 1))

# limit buy BTC/EUR, you pay €2500 and receive ฿1  when the order is closed
print(exchange.id, exchange.create_limit_buy_order('BTC/EUR', 1, 2500.00))

# pass/redefine custom exchange-specific order params: type, amount, price, flags, etc...
polo.create_market_buy_order('BTC/USD', 1, {'trading_agreement': 'agree'})